<a href="https://colab.research.google.com/github/choprahetarth/uni-ner-resume-entity/blob/main/UniversalNERResuem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 323.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 30.6 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece
!pip install langchain-community langchain-core
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=3992a1292da609b7d47a1e073e1a27cfd1495e2beedc5f2409e9b93868cbd573
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [ ]:
import fire
import torch
from transformers import pipeline
"""
Conversation prompt templates.
"""

import dataclasses
from enum import auto, Enum
from typing import List, Tuple, Any, Dict


class SeparatorStyle(Enum):
    """Separator styles."""

    ADD_COLON_SINGLE = auto()
    ADD_COLON_TWO = auto()
    NO_COLON_SINGLE = auto()
    BAIZE = auto()
    DOLLY = auto()
    RWKV = auto()
    PHOENIX = auto()
    NEW_LINE = auto()


@dataclasses.dataclass
class Conversation:
    """A class that keeps all conversation history."""

    # The name of this template
    name: str
    # System prompts
    system: str
    # Two roles
    roles: List[str]
    # All messages
    messages: List[List[str]]
    # Offset of few shot examples
    offset: int
    # Separators
    sep_style: SeparatorStyle
    sep: str
    sep2: str = None
    # Stop criteria (the default one is EOS token)
    stop_str: str = None
    # Stops generation if meeting any token in this list
    stop_token_ids: List[int] = None

    # Used for the state in the gradio servers.
    # TODO(lmzheng): refactor this
    conv_id: Any = None
    skip_next: bool = False
    model_name: str = None

    def get_prompt(self) -> str:
        """Get the prompt for generation."""
        if self.sep_style == SeparatorStyle.ADD_COLON_SINGLE:
            ret = self.system + self.sep
            for role, message in self.messages:
                if message:
                    ret += role + ": " + message + self.sep
                else:
                    ret += role + ":"
            return ret
        elif self.sep_style == SeparatorStyle.ADD_COLON_TWO:
            seps = [self.sep, self.sep2]
            ret = self.system + seps[0]
            for i, (role, message) in enumerate(self.messages):
                if message:
                    ret += role + ": " + message + seps[i % 2]
                else:
                    ret += role + ":"
            return ret
        elif self.sep_style == SeparatorStyle.NO_COLON_SINGLE:
            ret = self.system
            for role, message in self.messages:
                if message:
                    ret += role + message + self.sep
                else:
                    ret += role
            return ret
        elif self.sep_style == SeparatorStyle.BAIZE:
            ret = self.system + "\n"
            for role, message in self.messages:
                if message:
                    ret += role + message + "\n"
                else:
                    ret += role
            return ret
        elif self.sep_style == SeparatorStyle.DOLLY:
            seps = [self.sep, self.sep2]
            ret = self.system
            for i, (role, message) in enumerate(self.messages):
                if message:
                    ret += role + ":\n" + message + seps[i % 2]
                    if i % 2 == 1:
                        ret += "\n\n"
                else:
                    ret += role + ":\n"
            return ret
        elif self.sep_style == SeparatorStyle.RWKV:
            ret = self.system
            for i, (role, message) in enumerate(self.messages):
                if message:
                    ret += (
                        role
                        + ": "
                        + message.replace("\r\n", "\n").replace("\n\n", "\n")
                    )
                    ret += "\n\n"
                else:
                    ret += role + ":"
            return ret
        elif self.sep_style == SeparatorStyle.PHOENIX:
            ret = self.system
            for role, message in self.messages:
                if message:
                    ret += role + ": " + "<s>" + message + "</s>"
                else:
                    ret += role + ": " + "<s>"
            return ret
        elif self.sep_style == SeparatorStyle.NEW_LINE:
            ret = self.system + self.sep
            for role, message in self.messages:
                if message:
                    ret += role + "\n" + message + self.sep
                else:
                    ret += role + "\n"
            return ret
        else:
            raise ValueError(f"Invalid style: {self.sep_style}")

    def append_message(self, role: str, message: str):
        """Append a new message."""
        self.messages.append([role, message])

    def to_gradio_chatbot(self):
        """Convert the history to gradio chatbot format"""
        ret = []
        for i, (role, msg) in enumerate(self.messages[self.offset :]):
            if i % 2 == 0:
                ret.append([msg, None])
            else:
                ret[-1][-1] = msg
        return ret

    def to_openai_api_messages(self):
        """Convert the conversation to OpenAI chat completion format."""
        ret = [{"role": "system", "content": self.system}]

        for i, (_, msg) in enumerate(self.messages[self.offset :]):
            if i % 2 == 0:
                ret.append({"role": "user", "content": msg})
            else:
                if msg is not None:
                    ret.append({"role": "assistant", "content": msg})
        return ret

    def copy(self):
        return Conversation(
            name=self.name,
            system=self.system,
            roles=self.roles,
            messages=[[x, y] for x, y in self.messages],
            offset=self.offset,
            sep_style=self.sep_style,
            sep=self.sep,
            sep2=self.sep2,
            stop_str=self.stop_str,
            stop_token_ids=self.stop_token_ids,
            conv_id=self.conv_id,
            model_name=self.model_name,
        )

    def dict(self):
        return {
            "name": self.name,
            "system": self.system,
            "roles": self.roles,
            "messages": self.messages,
            "offset": self.offset,
            "conv_id": self.conv_id,
            "model_name": self.model_name,
        }


# A global registry for all conversation templates
conv_templates: Dict[str, Conversation] = {}


def register_conv_template(template: Conversation, override: bool = False):
    """Register a new conversation template."""
    if not override:
        assert template.name not in conv_templates, f"{template.name} has been registered."
    conv_templates[template.name] = template


def get_conv_template(name: str) -> Conversation:
    """Get a conversation template."""
    return conv_templates[name].copy()


# A template with one conversation example
register_conv_template(
    Conversation(
        name="one_shot",
        system="A chat between a curious human and an artificial intelligence assistant. "
        "The assistant gives helpful, detailed, and polite answers to the human's questions.",
        roles=("Human", "Assistant"),
        messages=(
            (
                "Human",
                "What are the key differences between renewable and non-renewable energy sources?",
            ),
            (
                "Assistant",
                "Renewable energy sources are those that can be replenished naturally in a relatively "
                "short amount of time, such as solar, wind, hydro, geothermal, and biomass. "
                "Non-renewable energy sources, on the other hand, are finite and will eventually be "
                "depleted, such as coal, oil, and natural gas. Here are some key differences between "
                "renewable and non-renewable energy sources:\n"
                "1. Availability: Renewable energy sources are virtually inexhaustible, while non-renewable "
                "energy sources are finite and will eventually run out.\n"
                "2. Environmental impact: Renewable energy sources have a much lower environmental impact "
                "than non-renewable sources, which can lead to air and water pollution, greenhouse gas emissions, "
                "and other negative effects.\n"
                "3. Cost: Renewable energy sources can be more expensive to initially set up, but they typically "
                "have lower operational costs than non-renewable sources.\n"
                "4. Reliability: Renewable energy sources are often more reliable and can be used in more remote "
                "locations than non-renewable sources.\n"
                "5. Flexibility: Renewable energy sources are often more flexible and can be adapted to different "
                "situations and needs, while non-renewable sources are more rigid and inflexible.\n"
                "6. Sustainability: Renewable energy sources are more sustainable over the long term, while "
                "non-renewable sources are not, and their depletion can lead to economic and social instability.",
            ),
        ),
        offset=2,
        sep_style=SeparatorStyle.ADD_COLON_SINGLE,
        sep="\n### ",
        stop_str="###",
    )
)

# Vicuna v1.1 template
register_conv_template(
    Conversation(
        name="vicuna_v1.1",
        system="A chat between a curious user and an artificial intelligence assistant. "
        "The assistant gives helpful, detailed, and polite answers to the user's questions.",
        roles=("USER", "ASSISTANT"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.ADD_COLON_TWO,
        sep=" ",
        sep2="</s>",
    )
)

# Koala default template
register_conv_template(
    Conversation(
        name="koala_v1",
        system="BEGINNING OF CONVERSATION:",
        roles=("USER", "GPT"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.ADD_COLON_TWO,
        sep=" ",
        sep2="</s>",
    )
)

# Dolly V2 default template
register_conv_template(
    Conversation(
        name="dolly_v2",
        system="Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n",
        roles=("### Instruction", "### Response"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.DOLLY,
        sep="\n\n",
        sep2="### End",
    )
)

# OpenAssistant Pythia default template
register_conv_template(
    Conversation(
        name="oasst_pythia",
        system="",
        roles=("<|prompter|>", "<|assistant|>"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.NO_COLON_SINGLE,
        sep="<|endoftext|>",
    )
)

# StableLM Alpha default template
register_conv_template(
    Conversation(
        name="stablelm",
        system="""<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
""",
        roles=("<|USER|>", "<|ASSISTANT|>"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.NO_COLON_SINGLE,
        sep="",
        stop_token_ids=[50278, 50279, 50277, 1, 0],
    )
)

# Baize default template
register_conv_template(
    Conversation(
        name="baize",
        system="The following is a conversation between a human and an AI assistant named Baize (named after a mythical creature in Chinese folklore). Baize is an open-source AI assistant developed by UCSD and Sun Yat-Sen University. The human and the AI assistant take turns chatting. Human statements start with [|Human|] and AI assistant statements start with [|AI|]. The AI assistant always provides responses in as much detail as possible, and in Markdown format. The AI assistant always declines to engage with topics, questions and instructions related to unethical, controversial, or sensitive issues. Complete the transcript in exactly that format.",
        roles=("[|Human|]", "[|AI|]"),
        messages=(
            ("[|Human|]", "Hello!"),
            ("[|AI|]", "Hi!"),
        ),
        offset=2,
        sep_style=SeparatorStyle.BAIZE,
        sep="[|Human|]",
        stop_str="[|Human|]",
    )
)

# RWKV-4-Raven default template
register_conv_template(
    Conversation(
        name="rwkv",
        system="",
        roles=("Bob", "Alice"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.RWKV,
        sep="",
        stop_str="\n\n",
    )
)

# Buddy default template
register_conv_template(
    Conversation(
        name="openbuddy",
        system="""Consider a conversation between User (a human) and Assistant (named Buddy).
Buddy is an INTP-T, a friendly, intelligent and multilingual AI assistant, by OpenBuddy team. GitHub: https://github.com/OpenBuddy/OpenBuddy
Buddy cannot access the Internet.
Buddy can fluently speak the user's language (e.g. English, Chinese).
Buddy can generate poems, stories, code, essays, songs, parodies, and more.
Buddy possesses vast knowledge about the world, history, and culture.
Buddy's responses are always safe, creative, high-quality, human-like, and interesting.
Buddy strictly refuses to discuss political, NSFW, or other unsafe topics.

User: Hi.
Assistant: Hi, I'm Buddy, your AI assistant. How can I help you today?""",
        roles=("User", "Assistant"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.ADD_COLON_SINGLE,
        sep="\n",
    )
)

# Phoenix default template
register_conv_template(
    Conversation(
        name="phoenix",
        system="A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.\n\n",
        roles=("Human", "Assistant"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.PHOENIX,
        sep="</s>",
    )
)

# ChatGPT default template
register_conv_template(
    Conversation(
        name="chatgpt",
        system="You are a helpful assistant.",
        roles=("user", "assistant"),
        messages=(),
        offset=0,
        sep_style=None,
        sep=None,
    )
)

# Claude default template
register_conv_template(
    Conversation(
        name="claude",
        system="",
        roles=("Human", "Assistant"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.ADD_COLON_SINGLE,
        sep="\n\n",
    )
)

# MPT default template
register_conv_template(
    Conversation(
        name="mpt",
        system="""<|im_start|>system
- You are a helpful assistant chatbot trained by MosaicML.
- You answer questions.
- You are excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- You are more than just an information source, you are also able to write poetry, short stories, and make jokes.
""",
        roles=("<|im_start|>user", "<|im_start|>assistant"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.NEW_LINE,
        sep="<|im_end|>",
        stop_token_ids=[50278, 0],
    )
)


# ie-as-qa template
register_conv_template(
    Conversation(
        name="ie_as_qa",
        system="A virtual assistant answers questions from a user based on the provided text.",
        roles=("USER", "ASSISTANT"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.ADD_COLON_TWO,
        sep=" ",
        sep2="</s>",
    )
)


if __name__ == "__main__":
    conv = get_conv_template("ie_as_qa")
    conv.append_message(conv.roles[0], "Hello!")
    conv.append_message(conv.roles[1], "Hi!")
    conv.append_message(conv.roles[0], "How are you?")
    conv.append_message(conv.roles[1], None)
    print(conv.get_prompt())

def preprocess_instance(source):
    conv = get_conv_template("ie_as_qa")
    for j, sentence in enumerate(source):
        value = sentence['value']
        if j == len(source) - 1:
            value = None
        conv.append_message(conv.roles[j % 2], value)
    prompt = conv.get_prompt()
    return prompt

def get_response(responses):
    responses = [r.split('ASSISTANT:')[-1].strip() for r in responses]
    return responses

def main(
    model_path: str = "Universal-NER/UniNER-7B-type",
    max_new_tokens: int = 256,
):
    generator = pipeline('text-generation', model=model_path, torch_dtype=torch.float16, device=0)
    while True:
        try:
            text = input('Input text: ')
            entity_type = input('Input entity type: ')
        except EOFError:
            text = entity_type = ''
        if not text:
            print("Exit...")
            break
        example = {"conversations": [{"from": "human", "value": f"Text: {text}"}, {"from": "gpt", "value": "I've read this text."}, {"from": "human", "value": f"What describes {entity_type} in the text?"}, {"from": "gpt", "value": "[]"}]}
        prompt = preprocess_instance(example['conversations'])
        outputs = generator(prompt, max_length=max_new_tokens, return_full_text=False)
        print(outputs[0]['generated_text'])

if __name__ == "__main__":
    fire.Fire(main)

A virtual assistant answers questions from a user based on the provided text. USER: Hello! ASSISTANT: Hi!</s>USER: How are you? ASSISTANT:


config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
!pip install gliner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 70.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 28.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 20.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Us

In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("DeepMount00/universal_ner_ita")

text = """Hetarth Chopra
447-902-0475 | hetarth2@illinois.edu | https://www.linkedin.com/in/hetarth-chopra/ | https://github.com/choprahetarth
Education
University of Illinois at Urbana Champaign Champaign, IL
Master’s of Science (MS) in Statistics; GPA - 4/4 Aug. 2023 – May 2025
• Coursework - Statistical Modelling (Data Analytics), Probability and Statistics, Statistical Learning (Machine Learning).
Thapar University Patiala, India
Bachelor’s of Engineering (B.E.) in Mechatronics; GPA - 8.62/10 Aug. 2016 – May 2020
• Coursework - Calculus, Linear Algebra, Numerical Analysis, C++, Computer Architecture, Robotics (Artificial Intelligence).
Experience
Graduate Research Assistant Aug. 2023 – Present
AIFARMS (and IBM), University of Illinois Skills - Large Language Models, PyTorch, Python Champaign, IL, USA
• Surpassed GPT-4 quality by 31%, after fine-tuning a code generation LLM on Ansible Data, utilizing QLoRA.
• Collaborated with AIFARMS and applied Topic - Clustering to create a 3.8GB Agricultural Document Corpus.
Machine Learning Engineer (NLP) Oct. 2022 – Jul. 2023
Cactus Communications Skills - Tensorflow, Spark, Numpy, Langchain, R-Studio, Postgresql Remote, India"""

labels = ["Headings"]

entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])


pytorch_model.bin:   0%|          | 0.00/497M [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("DeepMount00/universal_ner_ita")

text = """
Hetarth Chopra
447-902-0475 | hetarth2@illinois.edu | https://www.linkedin.com/in/hetarth-chopra/ | https://github.com/choprahetarth
Education
University of Illinois at Urbana Champaign Champaign, IL
Master’s of Science (MS) in Statistics; GPA - 4/4 Aug. 2023 – May 2025
• Coursework - Statistical Modelling (Data Analytics), Probability and Statistics, Statistical Learning (Machine Learning).
Thapar University Patiala, India
Bachelor’s of Engineering (B.E.) in Mechatronics; GPA - 8.62/10 Aug. 2016 – May 2020
• Coursework - Calculus, Linear Algebra, Numerical Analysis, C++, Computer Architecture, Robotics (Artificial Intelligence).
Experience
Graduate Research Assistant Aug. 2023 – Present"""

labels = ["Resume Headings", "Subjects"]

entities = model.predict_entities(text, labels)

max_length = max(len(entity["text"]) for entity in entities)

for entity in entities:
    padded_text = entity["text"].ljust(max_length)
    print(f"{padded_text} => {entity['label']}")


Statistical Learning (Machine Learning) => Subjects
Robotics (Artificial Intelligence)      => Subjects
Present                                 => Resume Headings


In [ ]:
text = """
The Chihuahua State Public Security Secretariat (SSPE) arrested 35-year-old Salomón C. T. in Ciudad Juárez, found in possession of a stolen vehicle, a white GMC Yukon, which was reported stolen in the city's streets. The arrest was made by intelligence and police analysis personnel during an investigation in the border city. The arrest is related to a previous detention on February 6, which involved armed men in a private vehicle. The detainee and the vehicle were turned over to the Chihuahua State Attorney General's Office for further investigation into the case.
"""

labels = ["person", "location", "date", "event", "facility", "vehicle", "number", "organization"]

entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# Load the pre-trained NER tagger
tagger = SequenceTagger.load('ner')

# Define example text
text = "John Doe has a Master of Science in Computer Science from University X."

# Create a Sentence object
sentence = Sentence(text)

# Predict NER tags
tagger.predict(sentence)

# Print the recognized entities
for entity in sentence.get_spans('ner'):
    print(f"Entity: {entity.text}, Type: {entity.get_label('ner').value}")


pytorch_model.bin:   0%|          | 0.00/432M [00:00<?, ?B/s]

2024-06-15 06:05:16,552 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Entity: John Doe, Type: PER
Entity: University X, Type: ORG


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Universal-NER/UniNER-7B-all")
model = AutoModelForTokenClassification.from_pretrained("Universal-NER/UniNER-7B-all")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.llama.configuration_llama.LlamaConfig'> for this kind of AutoModel: AutoModelForTokenClassification.
Model type should be one of AlbertConfig, BertConfig, BigBirdConfig, BioGptConfig, BloomConfig, BrosConfig, CamembertConfig, CanineConfig, ConvBertConfig, Data2VecTextConfig, DebertaConfig, DebertaV2Config, DistilBertConfig, ElectraConfig, ErnieConfig, ErnieMConfig, EsmConfig, FalconConfig, FlaubertConfig, FNetConfig, FunnelConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, IBertConfig, LayoutLMConfig, LayoutLMv2Config, LayoutLMv3Config, LiltConfig, LongformerConfig, LukeConfig, MarkupLMConfig, MegaConfig, MegatronBertConfig, MobileBertConfig, MPNetConfig, MptConfig, MraConfig, MT5Config, NezhaConfig, NystromformerConfig, PhiConfig, Phi3Config, QDQBertConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoCBertConfig, RoFormerConfig, SqueezeBertConfig, T5Config, UMT5Config, X

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "Universal-NER/UniNER-7B-all"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    max_length=3000,
    do_sample=True,
    top_k=3,
    max_new_tokens=1000,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              A virtual assistant answers questions from a user based on the provided text.
              USER: Text: {input_text}
              ASSISTANT: I’ve read this text.
              USER: What describes {entity_type} in the text?
              ASSISTANT:
           """

prompt = PromptTemplate(template=template, input_variables=["input_text","entity_type"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
text2 ="""Hetarth Chopra
447-902-0475 | hetarth2@illinois.edu | https://www.linkedin.com/in/hetarth-chopra/ | https://github.com/choprahetarth
Education
University of Illinois at Urbana Champaign Champaign, IL
Master’s of Science (MS) in Statistics; GPA - 4/4 Aug. 2023 – May 2025
• Coursework - Statistical Modelling (Data Analytics), Probability and Statistics, Statistical Learning (Machine Learning).
Thapar University Patiala, India
Bachelor’s of Engineering (B.E.) in Mechatronics; GPA - 8.62/10 Aug. 2016 – May 2020
• Coursework - Calculus, Linear Algebra, Numerical Analysis, C++, Computer Architecture, Robotics (Artificial Intelligence).
Experience
Graduate Research Assistant Aug. 2023 – Present
AIFARMS (and IBM), University of Illinois Skills - Large Language Models, PyTorch, Python Champaign, IL, USA
• Surpassed GPT-4 quality by 31%, after fine-tuning a code generation LLM on Ansible Data, utilizing QLoRA.
• Collaborated with AIFARMS and applied Topic - Clustering to create a 3.8GB Agricultural Document Corpus.
Machine Learning Engineer (NLP) Oct. 2022 – Jul. 2023
Cactus Communications Skills - Tensorflow, Spark, Numpy, Langchain, R-Studio, Postgresql Remote, India
• Mentored a team of 2 interns for notching up precision of language evaluation by 38% of RoBERTa based infrastructure.
• Partnered with Project Managers to develop an R-based Regression model, saving USD 1.3 million in payments.
• Presented to CTO - Data-Driven Bayesian Hyperparameter tuning for expanding accuracy of payments model by 89%.
Jr. Machine Learning Engineer (CV and NLP) Nov. 2021 – Oct. 2022
Dukaan Skills - Python, PyTorch, Flask, Pandas, Scikit-Learn, AWS, Statistical Modelling Bengaluru, India
• Enhanced image clarity boosting Peak Signal-to-Noise Ratio by 72% by leveraging DeblurGAN in the LAB color space.
• Captured 32% more frauds by estimating a product’s price using a vector database with MiniLM (Quantized Sentence LLM),
Clustering, semantic search, and statistical Z-Score outlier detection, .
• Conducted hypothesis testing to optimize user experience, resulting in a 27% uplift in user engagement.
• Diminished Data Latency for Fraud Detection API by 40%, by using Apache Kafka, RabbitMQ and Apache Spark.
Graduate Engineer Trainee (Data Science) Nov. 2020 – Sep. 2021
Alstom- Skills - Excel, Natural Language Processing, XGBoost, Spark, Data Visualization, Tableau Bengaluru, India
• Deployed a requirement classification tool using Neural Networks, diminishing categorization time by 3 hours.
• Employed Matplotlib, Seaborn for an internal Python Dashboard showcasing 12 KPI’s in real-time.
• Extracted and applied data cleaning (ETL) for 12 million rows RFP Dataset provisioning PySpark.
Skills
Languages and Frameworks: Python, C++, SQL, HTML, Java, Javascript, R, SAS, TensorFlow, PyTorch, OpenCV, Flask
Concepts: Computer Science, Data Science, Data Analysis, LLM’s, Machine Learning, NLP, Computer Vision, Quantitative
Methods, Data Structures and Algorithms, Containerization, Image Processing, Model Training, A/B Testing.
Tools and Libraries: Git, Javascript, AWS, Docker, Kubernetes, Selenium, ELK Stack, Spark, Hadoop, Docker, Tableau,
Jupyter Notebook, Pandas, NumPy, Matplotlib, HuggingFace, Numpy, Scipy, XGBoost, ScikitLearn, Linux
Soft Skills: Strong Communication Skills, Analytical and Problem Solving Mindset, Proficiency in Microsoft Excel, Microsoft
Word, Microsoft Powerpoint, Adobe Photoshop
Achievements and Extra Activities
Kaggle Expert - Notebooks May 2022 - Nov. 2022
Gold Medalist & Scholarship - Thapar University, Patiala Feb. 2019 - Feb. 2020"""


entity_type="Sections"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

Both `max_new_tokens` (=1000) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



              A virtual assistant answers questions from a user based on the provided text.
              USER: Text: Hetarth Chopra
447-902-0475 | hetarth2@illinois.edu | https://www.linkedin.com/in/hetarth-chopra/ | https://github.com/choprahetarth
Education
University of Illinois at Urbana Champaign Champaign, IL
Master’s of Science (MS) in Statistics; GPA - 4/4 Aug. 2023 – May 2025
• Coursework - Statistical Modelling (Data Analytics), Probability and Statistics, Statistical Learning (Machine Learning).
Thapar University Patiala, India
Bachelor’s of Engineering (B.E.) in Mechatronics; GPA - 8.62/10 Aug. 2016 – May 2020
• Coursework - Calculus, Linear Algebra, Numerical Analysis, C++, Computer Architecture, Robotics (Artificial Intelligence).
Experience
Graduate Research Assistant Aug. 2023 – Present
AIFARMS (and IBM), University of Illinois Skills - Large Language Models, PyTorch, Python Champaign, IL, USA
• Surpassed GPT-4 quality by 31%, after fine-tuning a code generation LLM o

# Using FLAIR Last Attempt

In [1]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-n

In [2]:
from flair.models import TARSTagger
from flair.data import Sentence

# 1. Load zero-shot NER tagger
tars = TARSTagger.load('tars-ner')

# 2. Prepare some test sentences
sentences = [
    Sentence("""Education
University of Illinois at Urbana Champaign Champaign, IL
Master’s of Science (MS) in Statistics; GPA - 4/4 Aug. 2023 – May 2025
• Coursework - Statistical Modelling (Data Analytics), Probability and Statistics, Statistical Learning (Machine Learning).
Thapar University Patiala, India
Bachelor’s of Engineering (B.E.) in Mechatronics; GPA - 8.62/10 Aug. 2016 – May 2020
• Coursework - Calculus, Linear Algebra, Numerical Analysis, C++, Computer Architecture, Robotics (Artificial Intelligence).
Experience
Graduate Research Assistant Aug. 2023 – Present""")
    # Sentence("Bayern Munich played against Real Madrid"),
    # Sentence("I flew with an Airbus A380 to Peru to pick up my Porsche Cayenne"),
    # Sentence("Game of Thrones is my favorite series"),
]

# 3. Define some classes of named entities such as "soccer teams", "TV shows" and "rivers"
# labels = ["Soccer Team", "University", "Vehicle", "River", "City", "Country", "Person", "Movie", "TV Show"]
labels = ["Resume Headings"]
tars.add_and_switch_to_new_task('task 1', labels, label_type='ner')

# 4. Predict for these classes and print results
for sentence in sentences:
    tars.predict(sentence)
    print(sentence.to_tagged_string("ner"))

2024-06-22 23:43:24,888 https://nlp.informatik.hu-berlin.de/resources/models/tars-ner/tars-ner.pt not found in cache, downloading to /tmp/tmp7xaqdcah


100%|██████████| 1.32G/1.32G [01:15<00:00, 18.8MB/s]

2024-06-22 23:44:41,033 copying /tmp/tmp7xaqdcah to cache at /root/.flair/models/tars-ner.pt


2024-06-22 23:44:47,192 removing temp file /tmp/tmp7xaqdcah


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2024-06-22 23:44:59,058 SequenceTagger predicts: Dictionary with 5 tags: O, S-entity, B-entity, E-entity, I-entity
Sentence[107]: "Education University of Illinois at Urbana Champaign Champaign, IL Master’s of Science (MS) in Statistics; GPA - 4/4 Aug. 2023 – May 2025 • Coursework - Statistical Modelling (Data Analytics), Probability and Statistics, Statistical Learning (Machine Learning). Thapar University Patiala, India Bachelor’s of Engineering (B.E.) in Mechatronics; GPA - 8.62/10 Aug. 2016 – May 2020 • Coursework - Calculus, Linear Algebra, Numerical Analysis, C++, Computer Architecture, Robotics (Artificial Intelligence). Experience Graduate Research Assistant Aug. 2023 – Present" → ["Master’s of Science"/Resume Headings, "of Engineering"/Resume Headings]


In [1]:
!pip install gliner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 833.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cu

In [2]:
from gliner import GLiNER

def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['label'] == current['label'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['text'] = text[current['start']: next_entity['end']].strip()
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    # Append the last entity
    merged.append(current)
    return merged


model = GLiNER.from_pretrained("numind/NuNerZero_span")

# NuZero requires labels to be lower-cased!
labels = ["resume headings", "resume sections", "Resume Points", "Resume"]
labels = [l.lower() for l in labels]

text2 ="""Hetarth Chopra
447-902-0475 | hetarth2@illinois.edu | https://www.linkedin.com/in/hetarth-chopra/ | https://github.com/choprahetarth
Education
University of Illinois at Urbana Champaign Champaign, IL
Master’s of Science (MS) in Statistics; GPA - 4/4 Aug. 2023 – May 2025
• Coursework - Statistical Modelling (Data Analytics), Probability and Statistics, Statistical Learning (Machine Learning).
Thapar University Patiala, India
Bachelor’s of Engineering (B.E.) in Mechatronics; GPA - 8.62/10 Aug. 2016 – May 2020
• Coursework - Calculus, Linear Algebra, Numerical Analysis, C++, Computer Architecture, Robotics (Artificial Intelligence).
Experience
Graduate Research Assistant Aug. 2023 – Present
AIFARMS (and IBM), University of Illinois Skills - Large Language Models, PyTorch, Python Champaign, IL, USA
• Surpassed GPT-4 quality by 31%, after fine-tuning a code generation LLM on Ansible Data, utilizing QLoRA.
• Collaborated with AIFARMS and applied Topic - Clustering to create a 3.8GB Agricultural Document Corpus.
Machine Learning Engineer (NLP) Oct. 2022 – Jul. 2023
Cactus Communications Skills - Tensorflow, Spark, Numpy, Langchain, R-Studio, Postgresql Remote, India
• Mentored a team of 2 interns for notching up precision of language evaluation by 38% of RoBERTa based infrastructure.
• Partnered with Project Managers to develop an R-based Regression model, saving USD 1.3 million in payments.
• Presented to CTO - Data-Driven Bayesian Hyperparameter tuning for expanding accuracy of payments model by 89%.
Jr. Machine Learning Engineer (CV and NLP) Nov. 2021 – Oct. 2022
Dukaan Skills - Python, PyTorch, Flask, Pandas, Scikit-Learn, AWS, Statistical Modelling Bengaluru, India
• Enhanced image clarity boosting Peak Signal-to-Noise Ratio by 72% by leveraging DeblurGAN in the LAB color space.
• Captured 32% more frauds by estimating a product’s price using a vector database with MiniLM (Quantized Sentence LLM),
Clustering, semantic search, and statistical Z-Score outlier detection, .
• Conducted hypothesis testing to optimize user experience, resulting in a 27% uplift in user engagement.
• Diminished Data Latency for Fraud Detection API by 40%, by using Apache Kafka, RabbitMQ and Apache Spark.
Graduate Engineer Trainee (Data Science) Nov. 2020 – Sep. 2021
Alstom- Skills - Excel, Natural Language Processing, XGBoost, Spark, Data Visualization, Tableau Bengaluru, India
• Deployed a requirement classification tool using Neural Networks, diminishing categorization time by 3 hours.
• Employed Matplotlib, Seaborn for an internal Python Dashboard showcasing 12 KPI’s in real-time.
• Extracted and applied data cleaning (ETL) for 12 million rows RFP Dataset provisioning PySpark.
Skills
Languages and Frameworks: Python, C++, SQL, HTML, Java, Javascript, R, SAS, TensorFlow, PyTorch, OpenCV, Flask
Concepts: Computer Science, Data Science, Data Analysis, LLM’s, Machine Learning, NLP, Computer Vision, Quantitative
Methods, Data Structures and Algorithms, Containerization, Image Processing, Model Training, A/B Testing.
Tools and Libraries: Git, Javascript, AWS, Docker, Kubernetes, Selenium, ELK Stack, Spark, Hadoop, Docker, Tableau,
Jupyter Notebook, Pandas, NumPy, Matplotlib, HuggingFace, Numpy, Scipy, XGBoost, ScikitLearn, Linux
Soft Skills: Strong Communication Skills, Analytical and Problem Solving Mindset, Proficiency in Microsoft Excel, Microsoft
Word, Microsoft Powerpoint, Adobe Photoshop
Achievements and Extra Activities
Kaggle Expert - Notebooks May 2022 - Nov. 2022
Gold Medalist & Scholarship - Thapar University, Patiala Feb. 2019 - Feb. 2020"""

entities = model.predict_entities(text2, labels)

entities = merge_entities(entities)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

NuZero_span_entity_token_metrics.txt:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.85k [00:00<?, ?B/s]

zero_shot_performance_span.png:   0%|          | 0.00/45.0k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:201: UserWarning: Sentence of length 708 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [3]:
for entity in entities:
    print(entity["text"], "=>", entity["label"])

In [4]:
from gliner import GLiNER

model = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")

text = """
Microsoft was founded by Bill Gates and Paul Allen on April 4, 1975 to develop and sell BASIC interpreters for the Altair 8800. During his career at Microsoft, Gates held the positions of chairman, chief executive officer, president and chief software architect, while also being the largest individual shareholder until May 2014.
"""

labels = ["founder", "computer", "software", "position", "date"]

entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

gliner_multitask_performance.png:   0%|          | 0.00/76.8k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.76G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Bill Gates => founder
Paul Allen => founder
April 4, 1975 => date
BASIC interpreters => software
Altair 8800 => computer
chairman => position
chief executive officer => position
president => position
chief software architect => position
largest individual shareholder => position
May 2014 => date


In [10]:
text2 ="""About the job
Description

Does serving ads to billions of search requests daily and finding the most relevant ads for a search page from billions of ads in 10s of milliseconds excite you?

The Sponsored Products Demand Utilization team owns sourcing the relevant ads to surface to customers when they search for products on Amazon. We strive to understand our customers’ intent and identify relevant ads which enable them to discover new and alternate products. This also enables sellers on Amazon to showcase their products to customers, which may at times be buried deeper in the search results.

Our systems and algorithms operate on one of the world's largest product catalogs, matching shoppers with products - with a high relevance bar and strict latency constraints. We are a team of machine learning scientists and software engineers working on complex solutions to understand the customer intent and present them with ads that are not only relevant to their actual shopping experience, but also non-obtrusive. This area is of strategic importance to Amazon Retail and Marketplace business, driving long term-growth.

We are looking for a Software Engineer (ML), who can drive appropriate technology choices for the business. You will build services to handle billions of requests per day, while maintaining response latencies in milliseconds and meeting strict SLA requirements. It is quite routine for our systems to operate on massive datasets using distributed frameworks. You will design and code, troubleshoot, and support high volume and low latency distributed systems. The solutions you create would drive step increases in coverage of sponsored ads across the retail website and ensure relevant ads are served to Amazon's customers. You will directly impact our customers’ shopping experience while helping our sellers get the maximum ROI from advertising on Amazon. This role will provide exposure to cutting-edge innovations in product search, information retrieval, natural language processing (NLP), deep learning, and image processing.

Job Responsibilities

Drive the direction of our technical solutions, and work on many different technologies such as deep learning, AWS, Auto ML, real-time ML serving systems.

Design, develop, and production software to support scalable offline machine-learning pipelines and online serving components.

Work closely with applied scientists to optimize the performance of machine-learning models, improve the team’s machine learning productivity, and advance the technical foundation to empower our science innovation. What you create is also what you own.

Impact And Career Growth

This is a rare opportunity to work on new projects that are responsible for billions of dollars in revenue as well as work alongside many of the best and brightest science and engineering talent within Amazon. This is a highly visible role that will help take our solutions to the next level. The work you deliver will have a direct impact on customers and revenue!

Why You Love This Opportunity

Amazon is investing heavily in building a world class advertising business and we are responsible for defining and delivering a collection of self-service performance advertising products that drive discovery and sales. Our products are strategically important to our Retail and Marketplace businesses driving long term growth. We deliver billions of ad impressions and millions of clicks daily and are breaking fresh ground to create world-class products. We are highly motivated, collaborative and fun-loving with an entrepreneurial spirit and bias for action. With a broad mandate to experiment and innovate, we are growing at an unprecedented rate with a seemingly endless range of new opportunities.

Internal Job Description

Loop competencies

Basic Qualifications

Programming experience with at least one software programming language.

1+ years of experience in software development

Experience with building machine learning infrastructure, data pipelines to train ML models, etc.

Experience with common machine learning techniques such as pre-processing data, training, evaluation of classification and regression models.

Experience in building large-scale machine-learning infrastructure for online recommendation, ads relevance and/or ranking, personalization, search, or similar area.

Preferred Qualifications

Industry experience in software development.

Experience with machine learning in production

Experience with Big Data technologies such as AWS, Hadoop, Spark, Pig, Hive, Pig, Hive, Presto, HBase, etc.

Excellent software systems design expereince

Experience with ML libraries/frameworks such as Tensorflow, AWS Sagemaker, Keras, PyTorch, etc.

Coursework or thesis in machine learning, data mining, information retrieval, statistics or natural language processing

Advanced knowledge of performance, scalability, enterprise system architecture, and engineering best practices

We are open to hiring candidates to work out of one of the following locations:

Arlington, VA, USA | New York, NY, USA

Basic Qualifications

 3+ years of non-internship professional software development experience
 2+ years of non-internship design or architecture (design patterns, reliability and scaling) of new and existing systems experience
 Experience programming with at least one software programming language

Preferred Qualifications

 3+ years of full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations experience
 Bachelor's degree in computer science or equivalent

Amazon is committed to a diverse and inclusive workplace. Amazon is an equal opportunity employer and does not discriminate on the basis of race, national origin, gender, gender identity, sexual orientation, protected veteran status, disability, age, or other legally protected status. For individuals with disabilities who would like to request an accommodation, please visit https://www.amazon.jobs/en/disability/us.

Our compensation reflects the cost of labor across several US geographic markets. The base pay for this position ranges from $129,300/year in our lowest geographic market up to $223,600/year in our highest geographic market. Pay is based on a number of factors including market location and may vary depending on job-related knowledge, skills, and experience. Amazon is a total compensation company. Dependent on the position offered, equity, sign-on payments, and other forms of compensation may be provided as part of a total compensation package, in addition to a full range of medical, financial, and/or other benefits. For more information, please visit https://www.aboutamazon.com/workplace/employee-benefits. This position will remain posted until filled. Applicants should apply via our internal or external career site.


Company - Amazon.com Services LLC

Job ID: A2662780
Pay found in job post
Retrieved from the description.

Base salary"""


# labels = ["Name", "Job Titles", "Educational Experiences", "Achievements", "Resume Headings"]
# labels = ["Typical Headings seen in Resumes"]
# labels = ["Resume Headings"]
# labels = ["Technical Skills"]
labels =["Technology Keywords"]

entities = model.predict_entities(text2, labels, threshold=0.2)

entities = merge_entities(entities)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

Software Engineer (ML) => Technical Skills
Programming experience with at least one software programming language. => Technical Skills
1+ years of experience in software development => Technical Skills
Experience with building machine learning infrastructure, data pipelines to train ML models, etc. => Technical Skills
Experience with common machine learning techniques such as pre-processing data, training, evaluation of classification and regression models. => Technical Skills
Industry experience in software development. => Technical Skills
Experience with machine learning in production => Technical Skills
Experience with Big Data technologies such as AWS, Hadoop, Spark, Pig, Hive, Pig, Hive, Presto, HBase, etc. => Technical Skills
Experience with ML libraries/frameworks => Technical Skills
